# 5-1 层和块
## 自定义块
从编程的角度来看，块由类（class）表示。 **它的任何子类都必须定义一个将其输入转换为输出的前向传播函数(forward())**， 并且必须存储任何必需的参数。 注意，有些块不需要任何参数。 最后，为了计算梯度，块必须具有反向传播函数。

每个block（Class）必须提供的基本功能:
- 将输入数据作为其前向传播函数的参数。
- 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

- 计算其输出关于输入的梯度，可通过其反向传播函数进行访问(如果不是实现特殊的运算符，这里可以自动实现)

- 存储和访问前向传播计算所需的参数。

- 根据需要初始化模型参数。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
class MLP(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256,10)
    # 根据输入返回模型的输出
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [3]:
net = MLP()
x = torch.rand(2,20)
net(x) #这里的forward函数是自动调用的

tensor([[ 0.1049, -0.2223, -0.1704, -0.1491, -0.1620,  0.0377, -0.0991, -0.1230,
         -0.2482,  0.0553],
        [ 0.0706, -0.0811, -0.0398, -0.0071,  0.0384,  0.1277, -0.2024, -0.0832,
         -0.0261, -0.0901]], grad_fn=<AddmmBackward0>)

## 5.1.2 顺序块


In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)] = module
    

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [11]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2,20)
net(X)

tensor([[-0.1611, -0.0203, -0.0658, -0.0285, -0.0178, -0.0358, -0.2219,  0.0549,
          0.0152,  0.1173],
        [-0.2200,  0.1045,  0.0355,  0.0329, -0.0730, -0.1035, -0.3430,  0.0208,
          0.0131,  0.1720]], grad_fn=<AddmmBackward0>)